# 1. Import Required Lib

In [1]:
!nvidia-smi

Wed May 15 12:37:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:05:00.0 Off |                    0 |
| N/A   29C    P0    51W / 250W |      0MiB / 22919MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle

import pdb
from torch.utils.data import DataLoader
import logging
import itertools

import argparse
from torch import optim
import time
import os
import load_dataset_es_sample
from load_dataset_es_sample import train_val_load
from bleu_score import BLEU_SCORE

from load_moss_dataset import *

from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
%matplotlib inline
import math
import copy

In [2]:
COMPRESSION = 0.722606950073752

In [3]:
device = torch.device('cuda')

# 2. Build data loaders

In [4]:
class Test(Dataset):
    def __init__(self, df, val = False):
        self.df = df
        self.val = val
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        english = self.df.iloc[idx,:]['en_idized']
        en_len = self.df.iloc[idx,:]['en_len']

        if self.val:
            en_sum = self.df.iloc[idx,:]['ref0'].lower()
            en_data = self.df.iloc[idx,:]['en_data'].lower()
            return [english, en_len, en_data, en_sum]
        else:
            return [english, en_len]

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    


def convert_idx_2_sent(tensor, lang_obj):
    word_list = []
    for i in tensor:
        if i.item() not in set([PAD_IDX,EOS_token,SOS_token]):
            word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

def convert_id_list_2_sent(list_idx, lang_obj):
    word_list = []
    if type(list_idx) == list:
        for i in list_idx:
            if i not in set([EOS_token]):
                word_list.append(lang_obj.index2word[i])
    else:
        for i in list_idx:
            if i.item() not in set([EOS_token,SOS_token,PAD_IDX]):
                word_list.append(lang_obj.index2word[i.item()])
    return (' ').join(word_list)

In [7]:
print(os.getcwd())

/home/brs426/unsupervised_document_summarization


## load the correct language pkl and change the data path to the place where you store gigaword valid.article.filter.txt

In [6]:
test, en_lang, intermediate_lang = test_load(50, "./it_en_lang_obj.pkl", '/home/brs426/')
# _, _, _, _, _, en_vecs, pi_vecs = train_val_load(48, "", '../../../scratch/brs426', 100000, '../../../scratch/brs426/cc.en.300.vec', '../../../scratch/brs426/cc.es.300.vec')

Reading English: test
['input.txt' 'task1_ref0' 'task1_ref1' 'task1_ref2' 'task1_ref3'
 'task1_ref4' 'input_len' 'ref0_len' 'ref1_len' 'ref2_len' 'ref3_len'
 'ref4_len']
tokenizing
token to index mapping
Computing length


In [7]:
test.head()

,en_data,en_tokenized,en_idized,en_len,ref0,ref1,ref2,ref3,ref4
0,We believe that the European Union can make an...,"[we, believe, that, the, european, union, can,...","[53, 509, 17, 5, 30493, 3863, 46, 116, 42, 136...",41,"We believe the EU can make an impact , but onl...",but only if it speaks with one voice and if it...,We believe the European Union can make an impa...,"We believe the EU can make an impact , but onl...",the UN can make an impact but needs proper too...
1,The figures are worse in Africa and parts of Asia,"[the, figures, are, worse, in, africa, and, pa...","[5, 2190, 28, 2269, 12, 31342, 7, 961, 9, 4315...",11,It is worse in Africa and parts of Asia,The figures are worse in Africa and parts of Asia,The figures are worse in Africa and Asia,It is worse in Africa and parts of Asia,The numbers are worse in Africa and Asia
2,The euro is the tangible result of EMU .,"[the, euro, is, the, tangible, result, of, emu...","[5, 12265, 13, 5, 14792, 640, 9, 77210, 6, 1]",10,EMU results in the tangible Euro,The Euro is a tangible result .,Euro is a tangible result of EMU,EMU results in the tangible Euro,The Euro is the result of EMU
3,"Some years ago ,","[some, years, ago, ,]","[89, 128, 361, 4, 1]",5,"Some years ago ,",Years ago,Some years ago .,"Some years ago ,","Years ago ,"
4,I have always believed that it is vital that B...,"[i, have, always, believed, that, it, is, vita...","[202, 36, 256, 2719, 17, 23, 13, 4672, 17, 809...",24,I always that it is vital that Britain plays a...,It is vital that Great Britain play a committe...,I believe that Britain should be a central pla...,I always that it is vital that Britain plays a...,central player in the European Union


In [52]:
test.shape

(150, 9)

In [8]:
def vocab_collate_func_test(batch):
    return [torch.from_numpy(np.array(batch[0][0])).unsqueeze(0),
            torch.from_numpy(np.array(batch[0][1])).unsqueeze(0), batch[0][-1]]

In [9]:
bs_dict = {'test':1}
shuffle_dict = {'test':False}

transformed_dataset = {'test':Test(test, val= True)}

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs_dict[x], collate_fn=vocab_collate_func_test,
                    shuffle=shuffle_dict[x], num_workers=0) for x in ['test']}

# 3. Build Models

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size,n_layers, vecs=None, rnn_type = 'lstm', device = device):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # self.embedding = Embedding(input_size, embed_dim, PAD_IDX)
        # Load in fasttext vectors
        if vecs:
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vecs))
        else:
            self.embedding = Embedding(input_size, embed_dim, PAD_IDX)
        self.rnn_type =  rnn_type
        self.dropout_in = nn.Dropout(p = 0.1)
        self.n_layers = n_layers
        self.device = device
        if rnn_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size,batch_first=True,bidirectional=True, num_layers = self.n_layers, dropout = 0.2)
        elif rnn_type == 'lstm':
            self.rnn = LSTM(embed_dim, hidden_size, batch_first=True,bidirectional=True, num_layers = n_layers,dropout = 0.2)

    def forward(self, enc_inp, src_len):
        sorted_idx = torch.sort(src_len, descending=True)[1]
        orig_idx = torch.sort(sorted_idx)[1]
        embedded = self.embedding(enc_inp)
        bs = embedded.size(0)
        output = self.dropout_in(embedded)
        if self.rnn_type == 'gru':
            hidden =  self.initHidden(bs)
            sorted_output = output[sorted_idx]
            sorted_len = src_len[sorted_idx]
            packed_output = nn.utils.rnn.pack_padded_sequence(sorted_output, sorted_len.data.tolist(), batch_first = True)
            packed_outs, hiddden = self.rnn(packed_output,(hidden, c))
            hidden = hidden[:,orig_idx,:]
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
            output = output[orig_idx]
            hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            return output, hidden, hidden
        elif self.rnn_type == 'lstm':
            hidden, c = self.initHidden(bs)
            sorted_output = output[sorted_idx]
            sorted_len = src_len[sorted_idx]
            packed_output = nn.utils.rnn.pack_padded_sequence(sorted_output, sorted_len.data.tolist(), batch_first = True)
            packed_outs, (hiddden, c) = self.rnn(packed_output,(hidden, c))
            hidden = hidden[:,orig_idx,:]
            c = c[:,orig_idx,:]
            output, _ = nn.utils.rnn.pad_packed_sequence(packed_outs, padding_value=PAD_IDX, batch_first = True)
            output = output[orig_idx]
            c = c.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            hidden = hidden.view(self.n_layers, 2, bs, -1).transpose(1, 2).contiguous().view(self.n_layers, bs, -1)
            return output, hidden, c
        
    def initHidden(self,bs):
        if self.rnn_type == 'gru' :
            return torch.zeros(self.n_layers*2, bs, self.hidden_size).to(self.device)
        elif self.rnn_type == 'lstm':
            return torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device),torch.zeros(self.n_layers*2,bs,self.hidden_size).to(self.device)

class Attention_Module(nn.Module):
    def __init__(self, hidden_dim, output_dim, device = device):
        super(Attention_Module, self).__init__()
        self.l1 = Linear(hidden_dim, output_dim, bias = False)
        self.l2 = Linear(hidden_dim+output_dim, output_dim, bias =  False)
        self.device = device
        
    def forward(self, hidden, encoder_outs, src_lens):
        ''' hiddden: bsz x hidden_dim
        encoder_outs: bsz x sq_len x encoder dim (output_dim)
        src_lens: bsz
        
        x: bsz x output_dim
        attn_score: bsz x sq_len'''
        x = self.l1(hidden)
        att_score = (encoder_outs.transpose(0,1) * x.unsqueeze(0)).sum(dim = 2)
        seq_mask = sequence_mask(src_lens, max_len = max(src_lens).item(), device = self.device).transpose(0,1)
        masked_att = seq_mask*att_score
        masked_att[masked_att==0] = -1e10
        attn_scores = F.softmax(masked_att, dim=0)
        x = (attn_scores.unsqueeze(2) * encoder_outs.transpose(0,1)).sum(dim=0)
        x = torch.tanh(self.l2(torch.cat((x, hidden), dim=1)))
        return x, attn_scores
        
class AttentionDecoderRNN(nn.Module):
    def __init__(self, output_size, embed_dim, hidden_size, vecs=None, n_layers = 1, len_control=False, attention = True, device = device):
        super(AttentionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        encoder_output_size = hidden_size
        # self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        # Load in fasttext vectors
        if vecs:
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(vecs))
        else:
            self.embedding = Embedding(output_size, embed_dim, PAD_IDX)
        self.len_control = len_control
        
        # Create lenEmbedding if this is a length-controlled decoder
        if len_control:
            self.lenEmbedding = nn.Embedding(250, 100)
        
        
        self.dropout = nn.Dropout(p=0.1)
        self.n_layers = n_layers
        self.device = device
        self.att_layer = Attention_Module(self.hidden_size, encoder_output_size,self.device) if attention else None
        self.layers = nn.ModuleList([
            LSTMCell(
                input_size=embed_dim + self.hidden_size + 100 if ((layer == 0) and attention and len_control) else embed_dim + self.hidden_size if ((layer == 0) and attention) else embed_dim if layer == 0 else hidden_size,
                hidden_size=hidden_size,
            )
            for layer in range(self.n_layers)
        ])
        self.fc_out = nn.Linear(self.hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.no_len_control_ll = nn.Linear(hidden_size, hidden_size)
#         self.len_control_ll = nn.Linear(1536, 1024)
#         self.len_params = nn.Parameter(torch.rand(1,512))
        
    
    # Need to pass t
    def forward(self, dec_input, context_vector, prev_hiddens, prev_cs, encoder_outputs, src_len, tar_len, init=False):
        bsz = dec_input.size(0)
        output = self.embedding(dec_input)
        output = self.dropout(output)
        
        
        # Check if decoder is length controlled
        # Get length embeddings and concatenate to output
        if self.len_control:
            tar_len[tar_len < 0] = 0
            tar_embed = self.lenEmbedding(tar_len).unsqueeze(1)
            output = torch.cat((output, tar_embed), dim=2)
        
        # Remove lenInit len_control stuff
        if init:
            # Initialize hidden state
            prev_hiddens = self.no_len_control_ll(torch.mean(prev_hiddens, dim=1)).unsqueeze(0)
#             if len_control:
#                 prev_hiddens = self.len_control_ll(torch.cat((torch.mean(prev_hiddens, dim=1), \
#                                                          self.len_params.repeat(bsz, 1)*tar_len.unsqueeze(1).float()),dim=1)).unsqueeze(0)
#                 prev_hiddens = self.len_control_ll(torch.cat((torch.mean(prev_hiddens, dim=1), \
#                                                            self.len_params.repeat(bsz, 1)*tar_len),dim=1)).unsqueeze(0)
#             else:
#                 prev_hiddens = self.no_len_control_ll(torch.mean(prev_hiddens, dim=1)).unsqueeze(0)
            
#         print("decoder", prev_hiddens.size(), len_control)
#         print(dec_input.size(), prev_hiddens.size(), context_vector.size(), prev_cs.size(), len_control, init)
        if self.att_layer is not None:
            cated_input = torch.cat([output.squeeze(1),context_vector], dim = 1)
        else:
            cated_input = output.squeeze(1)
        new_hiddens = []
        new_cs = []
        for i, rnn in enumerate(self.layers):
            hidden, c = rnn(cated_input, (prev_hiddens[i], prev_cs[i]))
            cated_input = self.dropout(hidden)
            new_hiddens.append(hidden.unsqueeze(0))
            new_cs.append(c.unsqueeze(0))
        new_hiddens = torch.cat(new_hiddens, dim = 0)
        new_cs = torch.cat(new_cs, dim = 0)

        # apply attention using the last layer's hidden state
        # Issue here with lenEmbed implementation
        if self.att_layer is not None:
            out, attn_score = self.att_layer(hidden, encoder_outputs, src_len)
        else:
            out = hidden
            attn_score = None
        context_vec = out
        out = self.dropout(out)
        out_vocab = self.softmax(self.fc_out(out))
        
        # Return tar_len decremented if length control otherwise return nothing for target length
        if self.len_control:
            return out_vocab, context_vec, new_hiddens, new_cs, attn_score, tar_len - 1
        else:
            return out_vocab, context_vec, new_hiddens, new_cs, attn_score, None

# 4. Utilites functions, training and evaluation functions 

In [11]:
def Embedding(num_embeddings, embedding_dim, padding_idx):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.uniform_(m.weight, -0.1, 0.1)
    nn.init.constant_(m.weight[padding_idx], 0)
    return m


def LSTM(input_size, hidden_size, **kwargs):
    m = nn.LSTM(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size,**kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


def Linear(in_features, out_features, bias=True, dropout=0):
    """Linear layer (input: N x T x C)"""
    m = nn.Linear(in_features, out_features, bias=bias)
    m.weight.data.uniform_(-0.1, 0.1)
    if bias:
        m.bias.data.uniform_(-0.1, 0.1)
    return m

In [12]:
def sequence_mask(sequence_length, max_len=None, device = 'cuda'):
    if max_len is None:
        max_len = sequence_length.max().item()
    batch_size = sequence_length.size(0)
    seq_range = torch.arange(0, max_len).long()
    seq_range_expand = seq_range.unsqueeze(0).repeat([batch_size,1])
    seq_range_expand = seq_range_expand.to(device)
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return (seq_range_expand < seq_length_expand).float()

In [15]:
def flatten_cel_loss(input,target,nll):
    input = input.transpose(1,2)
    bs, sl = input.size()[:2]
    return nll(input.contiguous().view(bs*sl,-1),target.contiguous().view(-1))

## change the embeding size and hidden size to your setting

In [17]:
# If translating from english -> pivot, no length control
encoder_w_att1 = EncoderRNN(en_lang.n_words,300,300,1, vecs=en_lang.vecs).to(device)
decoder_w_att1 = AttentionDecoderRNN(intermediate_lang.n_words,300,600, vecs=intermediate_lang.vecs,n_layers=1, len_control=False, attention = True).to(device)
 # From pivot -> english, use length control
encoder_w_att2 = EncoderRNN(intermediate_lang.n_words,300,300,1, vecs=intermediate_lang.vecs).to(device)
decoder_w_att2 = AttentionDecoderRNN(en_lang.n_words,300,600, vecs=en_lang.vecs, n_layers=1, len_control=True, attention=True).to(device)

In [41]:
print(en_lang.n_words, intermediate_lang.n_words)

112418 125827


## change encoder decoder path

In [18]:
encoder_w_att1.load_state_dict(torch.load('./english_italian_sample_encoder1.pth'))
decoder_w_att1.load_state_dict(torch.load('./english_italian_sample_decoder1.pth'))
encoder_w_att2.load_state_dict(torch.load('./italian_english_sample_encoder1.pth'))
decoder_w_att2.load_state_dict(torch.load('./italian_english_sample_decoder1.pth'))

In [19]:
def produce_out(encoder, decoder, data, lang_first, lang_second, m_type, verbose = False, replace_unk = False, len_cont = False):
    encoder.eval()
    decoder.eval()
    pred_corpus = []
    true_corpus = []
    src_corpus = []

    encoder_i = data[0].to(device)
    src_len = data[2].to(device)
    
    tar_len = data[3].to(device)
    bs,sl = encoder_i.size()[:2]
    en_out,en_hid,en_c = encoder(encoder_i,src_len)
    max_src_len_batch = max(src_len).item()
    prev_hiddens = en_out
    prev_cs = en_c
    decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
    prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
    d_out = []
    init = True
    attention_scores = []
    for i in range(sl*2):
        out_vocab, prev_output,prev_hiddens, prev_cs, attention_score, tar_len = decoder(decoder_input,prev_output, \
                                                                                prev_hiddens, prev_cs, en_out,\
                                                                                src_len, tar_len, len_cont, init)
        init = False
        topv, topi = out_vocab.topk(1)
#             decoder_input = topi.squeeze().detach().view(-1,1)
        d_out.append(topi.item())
        decoder_input = topi.squeeze().detach().view(-1,1)
        if m_type == 'attention':
            attention_scores.append(attention_score.unsqueeze(-1))
        if topi.item() == EOS_token:
            break

    if verbose:
        print("True Sentence:",data[-1])
        print("Pred Sentence:", pred_sent)
        print('-*'*50)
    score = bl.corpus_bleu(pred_corpus,[true_corpus],lowercase=True)[0]
#     print(score)
    
    return d_out

In [20]:
def half(encoder1, decoder1, data, lang_obj=None, tar_len=None, len_cont = False, prediction = False):

    encoder_i = data[0].to(device)
#     print("ENCODER I SHAPE", encoder_i)
    src_len = data[1].to(device)
    if tar_len is not None:
        tar_len = torch.tensor([tar_len]).to(device)
#     tar_len = None
#     if compression_ratio !=None:
#         tar_len = torch.tensor([int(compression_ratio * src_len.item())]).to(device)
        
#     summary = data[-1]
    
    bs, sl = encoder_i.size()[:2]

    en_out, en_hid, en_c = encoder1(encoder_i,src_len)
    
    prev_hiddens = en_out
    prev_cs = en_c
    decoder_input = torch.tensor([[SOS_token]]*bs).to(device)
    prev_output = torch.zeros((bs, en_out.size(-1))).to(device)
    
    d_out = []
    init = True
    for i in range(2*sl):
        out_vocab, prev_output,prev_hiddens, prev_cs, attention_score, tar_len = decoder1(decoder_input,prev_output, \
                                                                                prev_hiddens, prev_cs, en_out,\
                                                                                src_len, tar_len, init)
        init = False
        topv, topi = out_vocab.topk(1)
        d_out.append(topi.item())
        decoder_input = topi.squeeze().detach().view(-1,1)

        if topi.item() == EOS_token:
            break
        
#     print(torch.Tensor(d_out).to(device))
#     print("GOT D_OUT")
    if prediction:
        pred_sent = convert_id_list_2_sent(d_out,lang_obj)
        return pred_sent
#     , summary
    else:
        return torch.from_numpy(np.array(d_out)).unsqueeze(0), torch.from_numpy(np.array(len(d_out))).unsqueeze(0)


In [21]:
def validation_new(encoder1, decoder1, encoder2, decoder2, val_dataloader, lang_first, lang_second, compression_ratio=None):
    
    
    encoder1.eval()
    decoder1.eval()
    encoder2.eval()
    decoder2.eval()
    
    pred_corpus = []
    true_corpus = []
    
    # Need to keep original src length
    # Pass this in along with compression ratio

    ii = 0
    for data in val_dataloader:
        print(ii)
        ii += 0
        og_src = data[1]
        if compression_ratio is not None:
            tar_len = int(og_src.item() * compression_ratio)
        idx_token2, src_len2 = half(encoder1, decoder1, data)
        input_2 = [idx_token2, src_len2]
        pred_sent = half(encoder2, decoder2, input_2, lang_first, tar_len, \
                         True, prediction = True)
        
        
        pred_corpus.append(pred_sent)
        true_corpus.append(data[-1])
    return pred_corpus, true_corpus


In [59]:
i = 0
for data in dataloader['test']:
    half(encoder_w_att1, decoder_w_att1, data, intermediate_lang, prediction=True)
    print(i)
    i += 1
#     print('\n')
#     half(encoder_w_att1, decoder_w_att1, data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


KeyboardInterrupt: 

In [47]:
data = next(iter(dataloader['test']))
print(data)
half(encoder_w_att1, decoder_w_att1, data, intermediate_lang, tar_len=10, prediction=True)

[tensor([[67539,   213,  4763, 72439,     2, 15086,    37,     5,     2,   138,
          1408, 12974, 11417,    21, 41642,     4,    44,    48,  6985,   138,
         12974,  2141,    16,     5,   964,     8,  4604,    29,     5,     2,
             2, 57084,     6,     1]]), tensor([34]), "injury leaves kwan 's olympic hopes in limbo"]
TAR LENGTH 10


'la comisión de la comisión de UNK UNK , UNK y UNK , UNK , UNK y UNK , UNK , UNK , UNK , UNK , UNK , UNK .'

In [22]:
pred_corpus, true_corpus = validation_new(encoder_w_att1, decoder_w_att1, encoder_w_att2, decoder_w_att2, dataloader['test'], \
              en_lang, intermediate_lang, compression_ratio=COMPRESSION)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [23]:
test['pred'] = pred_corpus
test.to_csv('italian_moss.csv', index=False)
df_ = pd.read_csv('italian_moss.csv', header=0)

In [25]:
df_.head(10)

,en_data,en_tokenized,en_idized,en_len,ref0,ref1,ref2,ref3,ref4,pred
0,We believe that the European Union can make an...,"['we', 'believe', 'that', 'the', 'european', '...","[53, 509, 17, 5, 30493, 3863, 46, 116, 42, 136...",41,"We believe the EU can make an impact , but onl...",but only if it speaks with one voice and if it...,We believe the European Union can make an impa...,"We believe the EU can make an impact , but onl...",the UN can make an impact but needs proper too...,we believe that the european union can make an...
1,The figures are worse in Africa and parts of Asia,"['the', 'figures', 'are', 'worse', 'in', 'afri...","[5, 2190, 28, 2269, 12, 31342, 7, 961, 9, 4315...",11,It is worse in Africa and parts of Asia,The figures are worse in Africa and parts of Asia,The figures are worse in Africa and Asia,It is worse in Africa and parts of Asia,The numbers are worse in Africa and Asia,the figures are worst enough .
2,The euro is the tangible result of EMU .,"['the', 'euro', 'is', 'the', 'tangible', 'resu...","[5, 12265, 13, 5, 14792, 640, 9, 77210, 6, 1]",10,EMU results in the tangible Euro,The Euro is a tangible result .,Euro is a tangible result of EMU,EMU results in the tangible Euro,The Euro is the result of EMU,the euro is the result .
3,"Some years ago ,","['some', 'years', 'ago', ',']","[89, 128, 361, 4, 1]",5,"Some years ago ,",Years ago,Some years ago .,"Some years ago ,","Years ago ,",some years
4,I have always believed that it is vital that B...,"['i', 'have', 'always', 'believed', 'that', 'i...","[202, 36, 256, 2719, 17, 23, 13, 4672, 17, 809...",24,I always that it is vital that Britain plays a...,It is vital that Great Britain play a committe...,I believe that Britain should be a central pla...,I always that it is vital that Britain plays a...,central player in the European Union,i have always believed that the united kingdom...
5,We call for the suspension of the European Uni...,"['we', 'call', 'for', 'the', 'suspension', 'of...","[53, 506, 16, 5, 6267, 9, 5, 30493, 3863, 2282...",13,Suspension of the EU fisheries agreement is ca...,We request suspension of the European Union ...,We call for suspension of the EU fisheries agr...,Suspension of the EU fisheries agreement is ca...,We want to suspend the European Union fisherie...,we are calling for the eu fisheries .
6,I did n't want to stand out in the crowd any m...,"['i', 'did', ""n't"", 'want', 'to', 'stand', 'ou...","[202, 190, 100342, 170, 8, 1316, 65, 12, 5, 31...",14,I did n’t want to stand out any more,I no longer chose to stand out publicly .,I do n't want to me noticed in a crowd .,I did n’t want to stand out any more,I want to blend in with the crowd .,i myself wish to express my support for .
7,The Europol Management Board comprises one rep...,"['the', 'europol', 'management', 'board', 'com...","[5, 100020, 895, 955, 6141, 58, 4243, 37, 208,...",26,The Europol Management Board comprises one rep...,The Europol Management Board has one represent...,The Europol Management Board comprises one rep...,The Europol Management Board comprises one rep...,The Europol magagement board comprises one rep...,the europol management committee includes a re...
8,The introduction of the euro is a milestone in...,"['the', 'introduction', 'of', 'the', 'euro', '...","[5, 4017, 9, 5, 12265, 13, 10, 13143, 12, 5, 6...",22,The Euro is a milestone in the development of ...,The monetary changeover and development of the...,The introduction of the Euro was a milestone .,The Euro is a milestone in the development of ...,the introduction of the Euro is a milestone in...,the introduction of the euro is a progress in ...
9,In to replace these chemicals with others that...,"['in', 'to', 'replace', 'these', 'chemicals', ...","[12, 8, 2644, 130, 6622, 22, 454, 17, 28, 57, ...",34,"This would entail replacing freezers , cooler ...",To replace these chemicals with others that ar...,To replace these chemicals with others more in...,"This would entail replacing freezers , cooler ...",r

In [51]:
print(true_corpus)

["injury leaves kwan 's olympic hopes in limbo", 'us business attacks tough immigration law', 'gm december sales fall ## percent', 'thousands of croatians celebrate before world cup slalom', "laura bush <unk> rice to attend sirleaf 's inauguration in liberia", 'top republican lobbyist pleads guilty to florida fraud', 'somalia rivals in compromise on seat of government', 'portuguese airport workers strike could ground flights on friday', 'maniche renews partnership with mourinho', "hollywood shores up support for ocean 's thirteen", 'canada recommends avoiding travel to nepal', 'ford executive sees weaker us auto sales in ####', "us welcomes chinese journalist 's release highlights freedom concerns", 'two bankers admit theft from city players', 'unwell sharon admitted to hospital early', 'us hybrid vehicle sales expect to more than triple in seven years', "korea 's seo headed to dodgers from mets", 'hollywood starlet lindsay lohan admits bulimia battle', 'us insists soldiers act with re

In [40]:
print(next(iter(dataloader['test'])))

[tensor([[    2,   297,  8896,     2,     2,  8023,    73,     4,     2,   122,
          2182,     2, 17375,    17,  2426,     9,   101,    36,  4950,   122,
             2,  1547,    22,     4,  1677,     5,  2081,    61,     4,     2,
          7727,  6740,     6,     1]]), tensor([34]), "injury leaves kwan 's olympic hopes in limbo"]


In [18]:
# from rouge import Rouge 
# rouge = Rouge()

In [19]:
# scores = rouge.get_scores(pred_corpus, true_corpus)

In [ ]:
# print(scores)